In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [2]:
abstracts = pd.read_csv('/home/grace/Downloads/ChemProt_Corpus/chemprot_training/chemprot_training/chemprot_training_abstracts.tsv', sep='\t', header=None)
abstracts.columns = ['pmid', 'title', 'abstract']

In [3]:
abstracts.head()

,pmid,title,abstract
0,16357751,Selective costimulation modulators: a novel approach for the treatment of rheumatoid arthritis.,"T cells have a central role in the orchestration of the immune pathways that contribute to the inflammation and joint destruction characteristic of rheumatoid arthritis (RA). The requirement for a dual signal for T-cell activation and the construction of a fusion protein that prevents engagement of the costimulatory molecules required for this activation has led to a new approach to RA therapy. This approach is mechanistically distinct from other currently used therapies; it targets events early rather than late in the immune cascade, and it results in immunomodulation rather than complete immunosuppression. The fusion protein abatacept is a selective costimulation modulator that avidly binds to the CD80/CD86 ligands on an antigen-presenting cell, resulting in the inability of these ligands to engage the CD28 receptor on the T cell. Abatacept dose-dependently reduces T-cell proliferation, serum concentrations of acute-phase reactants, and other markers of inflammation, including the production of rheumatoid factor by B cells. Recent studies have provided consistent evidence that treatment with abatacept results in a rapid onset of efficacy that is maintained over the course of treatment in patients with inadequate response to methotrexate and anti-tumor necrosis factor therapies. This efficacy includes patient-centered outcomes and radiographic measurement of disease progression. Abatacept has also demonstrated a very favorable safety profile to date. This article reviews the rationale for this therapeutic approach and highlights some of the recent studies that demonstrate the benefits obtained by using abatacept. This clinical experience indicates that abatacept is a significant addition to the therapeutic armamentarium for the management of patients with RA."
1,14967461,Emerging role of epidermal growth factor receptor inhibition in therapy for advanced malignancy: focus on NSCLC.,"Combination chemotherapy regimens have emerged as the standard approach in advanced non-small-cell lung cancer. Meta-analyses have demonstrated a 2-month increase in median survival after platinum-based therapy vs. best supportive care, and an absolute 10% improvement in the 1-year survival rate. Just as importantly, cytotoxic therapy has produced benefits in symptom control and quality of life. Newer agents, including the taxanes, vinorelbine, gemcitabine, and irinotecan, have expanded our therapeutic options in the treatment of advanced non-small-cell lung cancer. Despite their contributions, we have reached a therapeutic plateau, with response rates seldom exceeding 30-40% in cooperative group studies and 1-year survival rates stable between 30% and 40%. It is doubtful that substituting one agent for another in various combinations will lead to any further improvement in these rates. The thrust of current research has focused on targeted therapy, and epidermal growth factor receptor inhibition is one of the most promising clinical strategies. Epidermal growth factor receptor inhibitors currently under investigation include the small molecules gefitinib (Iressa, ZD1839) and erlotinib (Tarceva, OSI-774), as well as monoclonal antibodies such as cetuximab (IMC-225, Erbitux). Agents that have only begun to undergo clinical evaluation include CI-1033, an irreversible pan-erbB tyrosine kinase inhibitor, and PKI166 and GW572016, both examples of dual kinase inhibitors (inhibiting epidermal growth factor receptor and Her2). Preclinical models have demonstrated synergy for all these agents in combination with either chemotherapy or radiotherapy, leading to great enthusiasm regarding their ultimate contribution to lung cancer therapy. However, serious clinical challenges persist. These include the identification of the optimal dose(s); the proper integration of these agents into popular, established cytotoxic regimens; and the 

In [4]:
abstracts.shape

(1020, 3)

In [5]:
entities = pd.read_csv('/home/grace/Downloads/ChemProt_Corpus/chemprot_training/chemprot_training/chemprot_training_entities.tsv', sep='\t', header=None)
# entity term (CHEMICAL, GENE-Y, GENE-N=mention that cannot be normalized to a database identifier)
entities.columns = ['pmid', 'entity_num', 'entity_term', 'start', 'end', 'mention']

In [6]:
entities.head()

,pmid,entity_num,entity_term,start,end,mention
0,11319232,T1,CHEMICAL,242,251,acyl-CoAs
1,11319232,T2,CHEMICAL,1193,1201,triacsin
2,11319232,T3,CHEMICAL,1441,1448,sucrose
3,11319232,T4,CHEMICAL,1637,1652,triacylglycerol
4,11319232,T5,CHEMICAL,1702,1711,acyl-CoAs


In [7]:
sample_x = abstracts.loc[abstracts.pmid==11319232]

In [8]:
x_t = sample_x.title.tolist()[0]
x = sample_x.abstract.tolist()[0]
# x = x.replace('(', ' ( ').replace(')', ' ) ')
x = x_t + ' ' + x

In [9]:
len(x)

1836

In [10]:
x_arr = x.split(' ')
len(x_arr)

267

In [11]:
x_df = pd.DataFrame(x_arr)
x_df.columns = ['content']
x_df['length'] = x_df.content.str.len()
x_df.head()

,content,length
0,Acyl-CoA,8
1,synthetase,10
2,isoforms,8
3,"1,",2
4,"4,",2


In [53]:
x_df['start_index'] = 0
x_df['t_length'] = x_df.length
for ind, row in x_df.iterrows():
    if ind >0:
        x_df['t_length'].iloc[ind] = row['length']+x_df.iloc[ind-1]['t_length']+1
        x_df['start_index'].iloc[ind] = x_df['t_length'].iloc[ind] - row['length']

/home/grace/.local/lib/python3.5/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [55]:
x_df.tail(10)

,content,length,t_length,start_index
257,metabolic,9,1763,1754
258,pathways,8,1772,1764
259,through,7,1780,1773
260,different,9,1790,1781
261,ACS,3,1794,1791
262,isoforms,8,1803,1795
263,in,2,1806,1804
264,unique,6,1813,1807
265,subcellular,11,1825,1814
266,locations.,10,1836,1826


In [56]:
y = entities.loc[entities.pmid==11319232]
y['length'] = y.mention.str.len()
y.tail()
y[y.mention=='ACS']
# for ind, row in y.sort_values(by=['length'], ascending=False).iterrows():
#     print(row['mention'])

/home/grace/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,pmid,entity_num,entity_term,start,end,mention,length
10,11319232,T11,GENE-N,1212,1215,ACS,3
20,11319232,T21,GENE-N,1791,1794,ACS,3
29,11319232,T30,GENE-N,197,200,ACS,3
31,11319232,T32,GENE-N,840,843,ACS,3
33,11319232,T34,GENE-N,947,950,ACS,3
36,11319232,T37,GENE-N,1081,1084,ACS,3


In [ ]:
# x_df[(x_df.start_index==1081) & (x_df.t_length==1084)]

In [77]:
x_df['label'] = 'O'
y = y.sort_values(by=['length'], ascending=False)
for ind, row in y.iterrows():
    print(row['mention'])
    print(row['start'], row['end'])
    mentions = x_df[(x_df.start_index>=row['start']) & (x_df.t_length<=row['end'])]
    print(mentions)
    ii = 0
    for ind_m, m in mentions.iterrows():
        if x_df['label'].iloc[ind_m]=='O':
            if ii==0: 
                x_df['label'].iloc[ind_m] = 'B-'+row['entity_term']
            else:
                x_df['label'].iloc[ind_m] = 'I-'+row['entity_term']
        ii += 1

Acyl-CoA synthetase isoforms 1, 4, and 5
0 40
      content  length  t_length  start_index label
0  Acyl-CoA    8       8         0            O   
1  synthetase  10      19        9            O   
2  isoforms    8       28        20           O   
3  1,          2       31        29           O   
4  4,          2       34        32           O   
5  and         3       38        35           O   
6  5           1       40        39           O   
acyl-CoA synthetase
176 195
       content  length  t_length  start_index label
26  acyl-CoA    8       184       176          O   
27  synthetase  10      195       185          O   
N-ethylmaleimide
790 806
Empty DataFrame
Columns: [content, length, t_length, start_index, label]
Index: []
triacylglycerol
1637 1652
             content  length  t_length  start_index label
242  triacylglycerol  15      1652      1637         O   
Troglitazone
898 910
Empty DataFrame
Columns: [content, length, t_length, start_index, label]
Index: []
Triacsin

/home/grace/.local/lib/python3.5/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)



ACS5
1260 1264
    content  length  t_length  start_index label
179  ACS5    4       1264      1260         O   
ACS4
1369 1373
    content  length  t_length  start_index label
197  ACS4    4       1373      1369         O   
ACS5
1402 1406
Empty DataFrame
Columns: [content, length, t_length, start_index, label]
Index: []
ACS1
1496 1500
    content  length  t_length  start_index label
221  ACS1    4       1500      1496         O   
ACSs
372 376
   content  length  t_length  start_index label
56  ACSs    4       376       372          O   
ACS4
1505 1509
    content  length  t_length  start_index label
223  ACS4    4       1509      1505         O   
ACS1
1606 1610
    content  length  t_length  start_index label
235  ACS1    4       1610      1606         O   
ACS5
525 529
Empty DataFrame
Columns: [content, length, t_length, start_index, label]
Index: []
ACS1
509 513
Empty DataFrame
Columns: [content, length, t_length, start_index, label]
Index: []
ACS4
515 519
Empty DataFrame
Column

In [76]:
x_df.to_csv('./test.tsv', sep='\t')